In [1]:
# there'll be a direcory called videoMetaData/
# there'll be folders for each device?
# then there'll be folders for each kind of processing
#   time indexes, and why not, lets also put in file name and frame index
#   just poses for now
    # in the format time index, pose list object

# also for the cameras lets build them up one use case at a time
# general tracking multiclass classification
#   sitting - bedroom
#   standing
#   lying down
#   showering - bathroom
#   pooping
#   taking a shower
#   brushing teeth
#   putting corizone on
#   making food - kitchen
#   wiping nose - all

# later I'll have a different set of cameras to help with facial analysis
# I'll still have one recording I can test some processing strategies
# at linux computer

In [2]:
# let's brainstorm version 2 of this process
# all of the cameras are sending to a folder called recent captures
# for now I'll just run the intake scrip manually

# get the list of folder names

# for each folder
# get the device name from the folder name
# get a list of filenames without type suffixes

# for each of names
# generate the desired location of the video
# make sure the file location for the video is created
# move the video there

# these will be parquets with millions of lines
# honestly I don't want to be opening them all the time
# or even really leaving them open

# nahh let's split by day that's honestly better in so many ways than by month
# yeah the smallest partquets would be like 4k rows but that's fine
# now a question lets just leave the files named appropriately in the month folder
# instead of further in the day folders the videos are in
# name for the parquet will be classic DEVICENAME-YYYY-MM-DD.parquet
# the columns will be - timestamp - fileLocation(YYYY-MM/DD/DN_TS_TS.mp4) - frameNumberInFile
# i guess later on it would make sense to put more frame specific metadata like exposure



# a remider of the file structure
# videoProcessing
# workingData
# |___homeVideo
#     |___testCam
#         |___frameMetaData
#             |___TS_TS.parquet
#             |___...
# videoData
# |___testCam
#     |___testCam_YYYY-MM(-)ZZZZ
#         |___testCam_YYYY-MM-DD(-)ZZZZ/
#             |___testCam_TS_TS.mp4
#             |___...
# recentCaptures
# |___testCam_YYYY-MM-DD(-)ZZZZ/
#     |___testCam_TS_TS.mp4
#     |___testCam_TS_TS.parquet
#     |___...

# let's move the folder from /home/uploadingGuest/recentCaptures/
# before running this code



In [1]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("videoProcessing")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + 'videoProcessing/')
from utils import writeWorkingDf

import os
import shutil
import pandas as pd
#get list of folders in recent captures
basePath = "/home/chowder/Documents/"
rcFolders = os.listdir(basePath + "recentCaptures/")

# for each folder
for folderName in sorted(rcFolders):
    deviceName = folderName.split("_")[0]
    tzOffset = folderName[-5:]
    startTs = folderName.split("_")[1]
    monthFolderName = deviceName + "_" + "-".join(startTs.split("-")[:2]) + tzOffset
    baseTarget =  basePath + "/".join(['videoData', deviceName, monthFolderName, ""])

    newDayFolder = basePath + "recentCaptures/" + folderName + "/"
    fileIdentifiers = list(set([f.split(".")[0] for f in os.listdir(newDayFolder)]))

    if not os.path.exists(baseTarget + folderName + "/"):
        os.makedirs(baseTarget + folderName + "/")
    
    #move mp4's
    for fid in fileIdentifiers:
        source = newDayFolder + fid + ".mp4"
        destination = baseTarget + folderName + "/" + fid + ".mp4"
        shutil.move(source, destination)
    
    print(f"moved all mp4s for {folderName}")

    # generate a parquet to add
    for i, fid in enumerate(sorted(fileIdentifiers)):
        source = newDayFolder + fid + ".parquet"
        readDf = pd.read_parquet(source)
        times = readDf.index.to_list() #change this to index.to_list() after the change
        
        dataToAdd = {}
        dataToAdd['sampleDT'] = times
        dataToAdd['frameNum'] = [x for x in range(len(times))]
        fileLocation = "/".join([deviceName, monthFolderName, folderName, (fid+".mp4")])
        dataToAdd['mp4Location'] = [fileLocation] * len(times)
        
        if i == 0:
            dayDf = pd.DataFrame(dataToAdd).set_index("sampleDT")
        else:
            dayDf = pd.concat([dayDf, pd.DataFrame(dataToAdd).set_index("sampleDT")])

        os.remove(source)

    dayDf = dayDf.sort_index()
    # write parquet
    destination = "homeVideo/" + deviceName + "/frameMetaData/"
    # print(dayDf.head(1))
    writeWorkingDf(destination, dayDf)
    print(f"wrote parquet for {folderName}")
    
    os.rmdir(newDayFolder)


moved all mp4s for bathroomCam_2024-12-04-0700
saved to a file named 2024-11-15T190802-0700_2024-12-04T235945-0700.parquet.gzip
in /home/chowder/Documents/workingData/homeVideo/bathroomCam/frameMetaData/
wrote parquet for bathroomCam_2024-12-04-0700
moved all mp4s for bathroomCam_2024-12-05-0700
saved to a file named 2024-11-15T190802-0700_2024-12-05T235945-0700.parquet.gzip
in /home/chowder/Documents/workingData/homeVideo/bathroomCam/frameMetaData/
wrote parquet for bathroomCam_2024-12-05-0700
moved all mp4s for bathroomCam_2024-12-06-0700
saved to a file named 2024-11-15T190802-0700_2024-12-06T235945-0700.parquet.gzip
in /home/chowder/Documents/workingData/homeVideo/bathroomCam/frameMetaData/
wrote parquet for bathroomCam_2024-12-06-0700
moved all mp4s for bathroomCam_2024-12-07-0700
saved to a file named 2024-11-15T190802-0700_2024-12-07T235945-0700.parquet.gzip
in /home/chowder/Documents/workingData/homeVideo/bathroomCam/frameMetaData/
wrote parquet for bathroomCam_2024-12-07-0700


In [2]:
dayDf

,frameNum,mp4Location
sampleDT,,
2024-11-18 10:03:45.005016-07:00,0,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-18 10:03:45.100172-07:00,1,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-18 10:03:45.200169-07:00,2,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-18 10:03:45.300163-07:00,3,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-18 10:03:45.400156-07:00,4,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
...,...,...
2024-11-18 11:10:41.700160-07:00,1795,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-18 11:10:41.800157-07:00,1796,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-18 11:10:41.900164-07:00,1797,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...


In [3]:
dayDf = dayDf.sort_index()
# write parquet
destination = "homeVideo/" + deviceName + "/frameMetaData/"
# print(dayDf.head(1))
writeWorkingDf(destination, dayDf)
print(f"wrote parquet for {folderName}")

saved to a file named 2024-11-14T222902-0700_2024-11-18T111042-0700.parquet.gzip
in /home/chowder/Documents/workingData/homeVideo/deskCam/frameMetaData/
wrote parquet for deskCam_2024-11-18-0700


In [ ]:
testDayLoc = "/home/chowder/Documents/rcbackup/bathroomCam_2024-11-14-0700/"
parquets = [x for x in os.listdir(testDayLoc) if x.split(".")[-1] == "parquet"]
parquets


['bathroomCam_2024-11-14T181915-001407-0700_2024-11-14T212815-000251-0700.parquet',
 'bathroomCam_2024-11-14T212830-000155-0700_2024-11-14T232300-000273-0700.parquet',
 'bathroomCam_2024-11-14T232315-000775-0700_2024-11-14T235945-000128-0700.parquet']

In [ ]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("videoProcessing")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + 'videoProcessing/')
from utils import getWorkingDf
loc = "homeVideo/deskCam/frameMetaData/"
df = getWorkingDf(loc)
df

,frameNum,mp4Location
sampleDT,,
2024-11-11 00:00:00.013893-07:00,0,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-11 00:00:00.100096-07:00,1,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-11 00:00:00.200088-07:00,2,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-11 00:00:00.300088-07:00,3,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-11 00:00:00.400083-07:00,4,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
...,...,...
2024-11-17 23:58:45.000252-07:00,933,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-17 23:59:00.000265-07:00,934,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...
2024-11-17 23:59:15.000252-07:00,935,deskCam/deskCam_2024-11-0700/deskCam_2024-11-1...


In [ ]:
df.sort_index()

,frameNum,mp4Location
sampleDT,,
2024-11-16 00:00:00.000330-07:00,0,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
2024-11-16 00:00:15.002624-07:00,1,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
2024-11-16 00:00:30.000245-07:00,2,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
2024-11-16 00:00:45.000130-07:00,3,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
2024-11-16 00:01:00.000275-07:00,4,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
...,...,...
2024-11-16 23:58:45.000131-07:00,899,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
2024-11-16 23:59:00.000133-07:00,900,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
2024-11-16 23:59:15.000132-07:00,901,bathroomCam/bathroomCam_2024-11-0700/bathroomC...
